## General information
This kernel constists many usefull features (in our opinion).   
Also this kernel need more then 16gb RAM while processing dataset (Didn't work at Kaggle).  
Part of the final solution was based on the following kernels:
- [Elo EDA and models](https://www.kaggle.com/artgor/elo-eda-and-models/)
- [Elo world](https://www.kaggle.com/fabiendaniel/elo-world)
- [Simple Exploration Notebook - Elo](https://www.kaggle.com/sudalairajkumar/simple-exploration-notebook-elo)  
![](https://storage.googleapis.com/kaggle-competitions/kaggle/10445/logos/thumb76_76.png?t=2018-10-24-17-14-05)

In [ ]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
import gc
import os

import re

from sklearn.model_selection import KFold, RepeatedKFold

import time
import lightgbm as lgb

from sklearn.metrics import mean_squared_error,roc_auc_score
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder,StandardScaler,MinMaxScaler,RobustScaler
from sklearn.model_selection import GridSearchCV
from catboost import CatBoostRegressor, Pool, CatBoost
from sklearn.linear_model import LinearRegression,BayesianRidge
from boruta import BorutaPy
#from keras.utils import to_categorical

import matplotlib.pyplot as plt 
import seaborn as sns 
from pylab import rcParams

import datetime

import warnings
warnings.filterwarnings('ignore')

plt.style.use('ggplot')

## Functions
Here you can see functions, that we used to change data format and clar/prepare it.
- <b>model_lgb</b> - simple lgbm model function
- <b>rmse</b> - RMSE metric calculation function
- <b>replace_id</b> - function for card_id replacement to number - it's usefull for the first iteration.
- <b>replace_month</b> - function to make some time related features like month_id and elapsed_time
- <b>dist_holiday</b> - get from [this kernel](https://www.kaggle.com/cttsai/simple-xgboost-cv-bagging)

In [ ]:
def rmse(y_true,y_pred):
    return np.sqrt(np.power(y_true-y_pred,2).sum()/len(y_true))

def replace_id(df,dictionary = False):
    
    df['card_id2']  = df.card_id.str.replace('C','')
    df['card_id2']  = df.card_id2.str.replace('_','')
    df['card_id2']  = df.card_id2.str.replace('I','')
    df['card_id2']  = df.card_id2.str.replace('D','')
    df['card_id2']  = df.card_id2.str.replace('a','10')
    df['card_id2']  = df.card_id2.str.replace('b','11')
    df['card_id2']  = df.card_id2.str.replace('c','12')
    df['card_id2']  = df.card_id2.str.replace('d','13')
    df['card_id2']  = df.card_id2.str.replace('e','14')
    df['card_id2']  = df.card_id2.str.replace('f','15').astype(np.float64)   
    
    if dictionary:
        d = df[['card_id','card_id2']]
        df.drop('card_id',axis=1,inplace=True)
        df = df.rename(columns = {'card_id2':'card_id'})
        return df,d
    df.drop('card_id',axis=1,inplace=True)
    df = df.rename(columns = {'card_id2':'card_id'})
    return df
    

def replace_month(df):
    df['month_id']  = df.first_active_month.str.replace("-",'').fillna(201802).astype(int)
    
    df['first_active_month'] = pd.to_datetime(df['first_active_month'])
    df['elapsed_time'] = (datetime.date(2018, 8, 1) - df['first_active_month'].dt.date).dt.days
    return df

def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

def dist_holiday(df, col_name, date_holiday, date_ref, period=100):
    df[col_name] = np.maximum(np.minimum((pd.to_datetime(date_holiday) - df[date_ref]).dt.days, period), 0)
    
def submit(df,name,score):
    arr = []
    path = '../'
    for a in os.listdir(path):
        arr.append(a)
    arr = [re.sub("[^0-9_]",'',a) for a in arr ]
    arr = [a.split('_') for a in arr if a not in ['']]
    arr = [int(a[0]) for a in arr if a[0] not in ['']]
    pre = str(max(arr))
    str_dat = str(time.localtime().tm_year)+'_'+str(time.localtime().tm_mon)+'_'+str(time.localtime().tm_mday)
    str_dat
    df[['card_id','target']].to_csv(path+pre+'_'+str_dat+'_CV_'+str(score)+'.csv')

### Deanon features
[This kernel](https://www.kaggle.com/raddar/towards-de-anonymizing-the-data-some-insights)

In [ ]:

def deanon_purchase(purchase):
    return np.round(purchase/0.00150265118 + 497.06,8)

def deanon_target(target):
    return np.exp2(target)

def anon_target(target):
    return np.log2(target)


## Load data

In [ ]:
%%time
train = pd.read_csv('../inpt/train.csv')
test_final = pd.read_csv('../input/test.csv')
hist = pd.read_csv('../input/historical_transactions.csv',parse_dates=["purchase_date"])
merch = pd.read_csv('../input/merchants.csv')
new_t = pd.read_csv('../input/new_merchant_transactions.csv',parse_dates=["purchase_date"])

In [ ]:
train = reduce_mem_usage(train)
test_final = reduce_mem_usage(test_final)
hist = reduce_mem_usage(hist)
merch = reduce_mem_usage(merch)
new_t = reduce_mem_usage(new_t)

In [ ]:
plt.hist(train['target']);

In [ ]:
print('Sample with target less then -33: ',train[train.target < -33.].card_id.nunique(),'rows')

Here we can see, that we have many target near zero and some strange target with value <b>-33.21928095</b>.  
For RMSE this values can be very important.  
I take this card_id into separate variable to use this in a future

In [ ]:
card_with_large_negative = train[train.target < -33.].card_id.unique()
print('Cards with large negative:',train[train.target < -33.].card_id.nunique())

## Feature engineering

In [ ]:
%%time
train = replace_month(train)
test_final = replace_month(test_final)
gc.collect()

### Merchant.csv
In many kernel we can see this file is useless, but let's try to get some features from it.

In [ ]:
merch.category_1 = merch.category_1.replace({'Y':'1','N':'0'}).astype(int)
merch.category_2.fillna(1,inplace=True)
merch.category_4 = merch.category_4.replace({'Y':'1','N':'0'}).astype(int)
merch.most_recent_sales_range = merch.most_recent_sales_range.replace({'A':'1','B':'2','C':'3','D':'4','E':'5'}).astype(int)
merch.most_recent_purchases_range = merch.most_recent_purchases_range.replace({'A':'1','B':'2','C':'3','D':'4','E':'5'}).astype(int)


Deanon numerical and purchase

In [ ]:
merch['numerical_1'] = np.round(merch['numerical_1'] / 0.009914905 + 5.79639, 0)
merch['numerical_2'] = np.round(merch['numerical_2'] / 0.009914905 + 5.79639, 0)
merch['avg_purchases_lag3' ] = deanon_purchase(merch['avg_purchases_lag3' ])
merch['avg_purchases_lag6' ] = deanon_purchase(merch['avg_purchases_lag6' ])
merch['avg_purchases_lag12'] = deanon_purchase(merch['avg_purchases_lag12'])

In [ ]:
merch.describe().T

Here we can see, that we have inf values in avg_purchases_lag3, avg_purchases_lag6, avg_purchases_lag12. Lets replace this on mean values.

In [ ]:
merch.loc[(merch.avg_purchases_lag3  == np.inf),'avg_purchases_lag3'] = merch[(merch.avg_purchases_lag3  != np.inf)]['avg_purchases_lag3'].mean()
merch.loc[(merch.avg_purchases_lag6  == np.inf),'avg_purchases_lag6'] = merch[(merch.avg_purchases_lag6  != np.inf)]['avg_purchases_lag6'].mean()
merch.loc[(merch.avg_purchases_lag12 == np.inf),'avg_purchases_lag12'] = merch[(merch.avg_purchases_lag12 != np.inf)]['avg_purchases_lag12'].mean()

In [ ]:
print(len(merch['merchant_id'].unique()))
print(merch['merchant_id'].count())

Also we can see, that some merchants have more then one record.  
Only 63 merchants have this problem, its less then 0.1% of data, and i just get mean for this samples

In [ ]:
merch = merch.groupby('merchant_id').mean().reset_index()
merch.columns = ['merch_'+a for a in merch.columns]
merch = merch.rename(columns={'merch_merchant_id':'merchant_id'})

After many test I get only this features, that may be good in feature importance

In [ ]:
merch = merch[['merchant_id','merch_avg_sales_lag3',
       'merch_avg_purchases_lag3','merch_numerical_1','merch_numerical_2']]

In [ ]:
hist  = pd.merge(hist ,merch,on='merchant_id',how='left')
new_t = pd.merge(new_t,merch,on='merchant_id',how='left')

In [ ]:
del merch;
gc.collect()

### Historical and new transactions

We split history data on two parts, because we can see in new transactions data only authorized purchases.  
Also i see on this kernel: [Simple Exploration Notebook - Elo](https://www.kaggle.com/sudalairajkumar/simple-exploration-notebook-elo) and think this is a good idea.  
plus in separate dataset i get card_ids, that have target with -33.21928095.

In [ ]:
auth = hist[hist['authorized_flag']== 'Y']
non_auth = hist[hist['authorized_flag']== 'N']
card_ln = hist[hist.card_id.isin(card_with_large_negative)]


#### This is the main function, that changes format historical_transaction and new_merchant_transactions 


In [ ]:
%%time
def format_transaction(added_1,pre=''):
    print('Starting formating for',pre)
    added_1['merchant_id'].fillna('M_ID_00a6ca8a8a',inplace=True) #one missing merchant
    orig = added_1.copy()
    #replace object data
    print('replace object data',end=' - ')
    added_1.authorized_flag = added_1.authorized_flag.replace({'Y':'1','N':'0'}).astype(int)
    added_1.category_1 = added_1.category_1.replace({'Y':'1','N':'0'}).astype(int)
    added_1.category_3 = added_1.category_3.replace({'A':'1','B':'2','C':'3',np.nan:'1'}).astype(int)
    print('ready')    
    #circled days, weeks and month
    print('Circle dates',end=' - ')
    added_1['cos_doy'  ] = np.cos(added_1.purchase_date.dt.dayofyear*2*np.pi/365)
    added_1['sin_doy'  ] = np.cos(added_1.purchase_date.dt.dayofyear*2*np.pi/365)    
    added_1['cos_dow'  ] = np.cos(added_1.purchase_date.dt.dayofweek*2*np.pi/7)
    added_1['sin_dow'  ] = np.cos(added_1.purchase_date.dt.dayofweek*2*np.pi/7)    
    added_1['cos_month'] = np.cos(added_1.purchase_date.dt.month*2*np.pi/12)
    added_1['sin_month'] = np.cos(added_1.purchase_date.dt.month*2*np.pi/12)    
    added_1['cos_week' ] = np.cos(added_1.purchase_date.dt.week*2*np.pi/52)
    added_1['sin_week' ] = np.cos(added_1.purchase_date.dt.week*2*np.pi/52)
    print('ready')
    #added year, month, and yearmonth
    print('added year, month, and yearmonth',end=' - ')
    added_1['year'] = added_1.purchase_date.dt.year.astype(str)
    added_1['weekofyear'] = added_1.purchase_date.dt.weekofyear
    added_1['dayofweek'] = added_1.purchase_date.dt.dayofweek
    added_1['month'] = added_1['purchase_date'].dt.month
    added_1['weekend'] = (added_1.purchase_date.dt.weekday >=5).astype(int)
    added_1['not_weekend'] = (added_1.purchase_date.dt.weekday <5).astype(int)
    added_1['hour'] = added_1.purchase_date.dt.hour  
    added_1.loc[added_1['month']<10,'month'] = '0'+added_1.loc[added_1['month']<10,'month'].astype(str)
    added_1['month_id'] = added_1['year'].astype(str)+added_1['month'].astype(str)
    
    added_1['month_diff']    = ((datetime.datetime.today() - added_1['purchase_date']).dt.days)//30
    added_1['month_diff']   += added_1['month_lag']  
    added_1['day']           = added_1['purchase_date'].dt.day
    added_1['dayofyear']     = added_1['purchase_date'].dt.dayofyear
    added_1['quarter']       = added_1['purchase_date'].dt.quarter
    
    added_1['price']              = added_1['purchase_amount'] / added_1['installments']
    added_1['duration']           = added_1['purchase_amount'] * added_1['month_diff']
    added_1['amount_month_ratio'] = added_1['purchase_amount'] / added_1['month_diff']
    
    print('ready')
    #add holidays    
    holidays = [
        ('Christmas_Day_2017', '2017-12-25'),  # Christmas: December 25 2017
        ('Mothers_Day_2017', '2017-06-04'),  # Mothers Day: May 14 2017
        ('fathers_day_2017', '2017-08-13'),  # fathers day: August 13 2017
        ('Children_day_2017', '2017-10-12'),  # Childrens day: October 12 2017
        ('Valentine_Day_2017', '2017-06-12'),  # Valentine's Day : 12th June, 2017
        ('Black_Friday_2017', '2017-11-24'),  # Black Friday: 24th November 2017
        ('Mothers_Day_2018', '2018-05-13'),
    ]
    
    for d_name, d_day in holidays:
        dist_holiday(added_1, d_name, d_day, 'purchase_date')
    
    #deanon purchase
    print('deanon purchase',end=' - ')
    added_1['purchase_amount'] = deanon_purchase(added_1['purchase_amount'])
    print('ready')
    #add category pairs
    print('add category pairs',end=' - ')
    added_1['category_12'] = added_1['category_1']*10 + added_1['category_2']
    added_1['category_23'] = added_1['category_2']*10 + added_1['category_3']
    added_1['category_13'] = added_1['category_1']*10 + added_1['category_3'] 
    
    added_1['category_all'] = added_1['category_1']*100 + added_1['category_2']*10 + added_1['category_3'] 
    
    added_1['month_diff'] = ((datetime.datetime.today() - added_1['purchase_date']).dt.days)//30
    added_1['month_diff'] += added_1['month_lag']
    
    added_1['month_diff_days'] = ((datetime.datetime.today() - added_1['purchase_date']).dt.days)//1
    added_1['month_diff_days'] += added_1['month_lag']*30.5
    
    added_1 = pd.get_dummies(added_1, columns=['category_3','category_2'])
    
    added_1.loc[:, 'purchase_date'] = pd.DatetimeIndex(added_1['purchase_date']).\
                                      astype(np.int64) * 1e-9    
    
    
    print('ready')    
    
    aggregate = {'installments':['sum','mean','median','std','var','nunique'],
                 'merchant_category_id':['nunique'],
                 'merchant_id':['nunique'],
                 'city_id':['nunique'],
                 'state_id':['nunique'],
                 'subsector_id':['nunique'],
                 
                 'purchase_amount': ['sum', 'mean','median', 'max', 'min', 'std','var'],
                 'authorized_flag':['sum'],
                 
                 'month_id':['nunique'],
                 'purchase_date':[np.ptp,'min','max'],
                 'month_lag':['max','min','mean','median','sum','std','var'],
                 'weekofyear':['max','min','mean','median','sum','std','var'],
                 'dayofweek':['max','min','mean','median','sum','std','var'],
                 'weekend':['sum','mean','median'],
                 'day':['sum','mean','median'],
                 'dayofyear':['sum','mean','median'],
                 'quarter':['sum','mean','median'],
                 'not_weekend':['sum','mean','median'],
                 'hour':['mean','median','sum'],
                 'month_diff':['mean','median','var','min','max'],     
                 'cos_doy'  :['mean','median','var','min','max'],    
                 'sin_doy'  :['mean','median','var','min','max'],    
                 'cos_dow'  :['mean','median','var','min','max'],    
                 'sin_dow'  :['mean','median','var','min','max'],    
                 'cos_month':['mean','median','var','min','max'],    
                 'sin_month':['mean','median','var','min','max'],    
                 'cos_week' :['mean','median','var','min','max'],    
                 'sin_week' :['mean','median','var','min','max'],    
                 
                 'category_1':['sum'],                 
                 'category_2_1.0': ['mean','median'],
                 'category_2_2.0': ['mean','median'],
                 'category_2_3.0': ['mean','median'],
                 'category_2_4.0': ['mean','median'],
                 'category_2_5.0': ['mean','median'],                 
                 'category_3_1': ['mean','median'],
                 'category_3_2': ['mean','median'],
                 'category_3_3': ['mean','median'],                 
                 'category_12':['mean','median'],
                 'category_23':['mean','median'],
                 'category_13':['mean','median'],                  
                 'category_all':['mean','median'],
                 'price': ['sum', 'mean','median', 'std','var'],           
                 'duration': ['sum', 'mean','median', 'std','var'],    
                 'amount_month_ratio': ['sum', 'mean','median', 'std','var'],
                 
                 'merch_avg_sales_lag3':['sum', 'mean','median', 'std','var'],
                 'merch_avg_purchases_lag3':['sum', 'mean','median', 'std','var'],
                 'merch_numerical_1':['sum', 'mean','median', 'std','var'],
                 'merch_numerical_2':['sum', 'mean','median', 'std','var'],
                 
                 'Christmas_Day_2017': ['mean', 'sum'],
                 'Mothers_Day_2017': ['mean', 'sum'],
                 'fathers_day_2017': ['mean', 'sum'],
                 'Children_day_2017': ['mean', 'sum'],
                 'Valentine_Day_2017': ['mean', 'sum'],
                 'Black_Friday_2017': ['mean', 'sum'],
                 'Mothers_Day_2018': ['mean', 'sum']
    }

    agg_month = {
                'purchase_amount': ['sum', 'mean','median', 'min', 'max', 'std','var'],
                'installments': ['sum','mean','median','nunique'],                
                'merch_avg_sales_lag3':['sum', 'mean','median', 'std'],
                'merch_avg_purchases_lag3':['sum', 'mean','median', 'std'],
                'merch_numerical_1':['sum', 'mean','median', 'std','var'],
                'merch_numerical_2':['sum', 'mean','median', 'std','var'],
                'weekofyear':['max','min','mean','median','sum','std','var'],
                'dayofweek':['max','min','mean','median','sum','std','var'],
                'weekend':['sum','mean','median'],
                'not_weekend':['sum','mean','median'],
                'hour':['mean','median','sum'],
                'day':['mean','median','sum']
                }
    
    agg_month2 = {
                'purchase_amount': ['sum', 'mean','median', 'min', 'max', 'std','var'],
                'installments': ['sum','mean','median','nunique'],                
                'merch_avg_sales_lag3':['sum', 'mean','median', 'std'],
                'merch_avg_purchases_lag3':['sum', 'mean','median', 'std'],   
                'merch_numerical_1':['sum', 'mean','median', 'std','var'],
                'merch_numerical_2':['sum', 'mean','median', 'std','var'],
                'weekofyear':['max','min','mean','median','sum','std','var'],
                'dayofweek':['max','min','mean','median','sum','std','var'],
                'weekend':['sum','mean','median'],
                'not_weekend':['sum','mean','median'],
                'hour':['mean','median','sum'],
                'month_diff':['mean','median'],
                'day':['mean','median','sum']
                }
    
    print('Start_aggr_month',end=' - ')
    #----
    #dont forget do it in cicle
    agg_month_1 = added_1.groupby(['card_id','month_lag']).agg(agg_month)
    agg_month_1.columns = ['ad1_'.join(col).strip() for col in agg_month_1.columns.values]
    agg_month_1.reset_index(inplace=True)

    agg_month_1 = agg_month_1.groupby(['card_id']).agg(['mean','median'])
    agg_month_1.columns = ['ad1_'.join(col).strip() for col in agg_month_1.columns.values]
    agg_month_1.reset_index(inplace=True)
    #----
    agg_month_2 = added_1.groupby(['card_id','month_diff']).agg(agg_month)
    agg_month_2.columns = ['ad2_'.join(col).strip() for col in agg_month_2.columns.values]
    agg_month_2.reset_index(inplace=True)

    agg_month_2 = agg_month_2.groupby(['card_id']).agg(['mean','median'])
    agg_month_2.columns = ['ad2_'.join(col).strip() for col in agg_month_2.columns.values]
    agg_month_2.reset_index(inplace=True)
    #----
    agg_month_3 = added_1.groupby(['card_id','installments']).agg(agg_month2)
    agg_month_3.columns = ['ad3_'.join(col).strip() for col in agg_month_3.columns.values]
    agg_month_3.reset_index(inplace=True)

    agg_month_3 = agg_month_3.groupby(['card_id']).agg(['mean','median'])
    agg_month_3.columns = ['ad3_'.join(col).strip() for col in agg_month_3.columns.values]
    agg_month_3.reset_index(inplace=True)
    
    #----
    agg_month_4 = added_1.groupby(['card_id','state_id']).agg(agg_month)
    agg_month_4.columns = ['ad4_'.join(col).strip() for col in agg_month_4.columns.values]
    agg_month_4.reset_index(inplace=True)

    agg_month_4 = agg_month_4.groupby(['card_id']).agg(['mean','median'])
    agg_month_4.columns = ['ad4_'.join(col).strip() for col in agg_month_4.columns.values]
    agg_month_4.reset_index(inplace=True)
    
    #----
    agg_month_5 = added_1.groupby(['card_id','category_all']).agg(agg_month)
    agg_month_5.columns = ['ad5_'.join(col).strip() for col in agg_month_5.columns.values]
    agg_month_5.reset_index(inplace=True)

    agg_month_5 = agg_month_5.groupby(['card_id']).agg(['mean','median'])
    agg_month_5.columns = ['ad5_'.join(col).strip() for col in agg_month_5.columns.values]
    agg_month_5.reset_index(inplace=True)
    print ('ready')
    
    print (added_1.columns)
    print('Start_aggr',end=' - ')
    added_1 = added_1.groupby(['card_id']).agg(aggregate)
    print ('ready')
    added_1.columns = [pre+'_'.join(col).strip() for col in added_1.columns.values]
    added_1.reset_index(inplace=True)
    print('Start merge',end=' - ')
    #added_1 = pd.merge(added_1,dummys_to_card,on='card_id',how='left')
    print ('ready')

    print('Start merge month',end=' - ')
    added_1 = pd.merge(added_1,agg_month_1,on='card_id',how='left')
    added_1 = pd.merge(added_1,agg_month_2,on='card_id',how='left')
    added_1 = pd.merge(added_1,agg_month_3,on='card_id',how='left')
    added_1 = pd.merge(added_1,agg_month_4,on='card_id',how='left')
    added_1 = pd.merge(added_1,agg_month_5,on='card_id',how='left')
    print ('ready')
    
    del agg_month_1,agg_month_2,agg_month_3,agg_month_4,agg_month_5;
    gc.collect();
    
    agr_feat = ['category_1','installments','state_id']
    
    #orig['category_all'] = orig['category_1']*100 + orig['category_2']*10 + orig['category_3'] 
    
    for a in agr_feat:
        print('Start gen features from '+a,end=' - ')
        agr = orig.groupby(['card_id',a])['purchase_amount'].mean()
        agr = pd.DataFrame(agr).reset_index().groupby('card_id')['purchase_amount'].agg(['mean','median'])
        agr.columns = [a+'_purchase_amount_'+col for col in agr.columns.values]
        agr.reset_index(inplace=True)
        added_1 = pd.merge(added_1,agr,on='card_id', how='left')
        print('ready')
        
    agr_feat = ['category_1','state_id']
    for a in agr_feat:
        print('Start gen features from '+a,end=' - ')
        agr = orig.groupby(['card_id',a])['installments'].mean()
        agr = pd.DataFrame(agr).reset_index().groupby('card_id')['installments'].agg(['mean','median'])
        agr.columns = [a+'_installments_'+col for col in agr.columns.values]
        agr.reset_index(inplace=True)
        added_1 = pd.merge(added_1,agr,on='card_id', how='left')
        print('ready')
    del orig;
    return added_1

### Formating all datasets

In [ ]:
%%time
added_2 = format_transaction(auth.copy(),'auth')
added_3 = format_transaction(non_auth.copy(),'non_auth')

Do the same with new merch

In [ ]:
%%time
added_4 = format_transaction(new_t.copy(),'new_')

![](http://)For card, that have target -33 we make simple features, just like borders.

In [ ]:
added_5 = card_ln.copy()
added_5.authorized_flag = added_5.authorized_flag.replace({'Y':'1','N':'0'}).astype(int)
added_5.category_1 = added_5.category_1.replace({'Y':'1','N':'0'}).astype(int)
added_5.category_3 = added_5.category_3.replace({'A':'1','B':'2','C':'3',np.nan:'4'}).astype(int)

added_5 = added_5.groupby(['card_id']).agg({'authorized_flag':'mean',
                                            'city_id':'nunique',
                                            'category_1':'mean',
                                            'installments':'mean',
                                            'category_3':'mean',
                                            'merchant_category_id':'nunique',
                                            'merchant_id':'nunique',
                                            'month_lag':'mean',
                                            'purchase_amount':'mean',
                                            'category_2':'mean',
                                            'state_id':'nunique',
                                            'subsector_id':'nunique'
    
}).reset_index()
added_5 = added_5.mean()
columns = added_5.reset_index().T.head(1).values[0]
added_5 = added_5.reset_index().T
added_5.columns = columns
# added_5 = added_5[added_5.card_id!='card_id']
added_5.columns = ['MEAN_LN_'+col for col in added_5.columns.values]

In [ ]:
LN_col = added_5.columns

### Lets join all feature to train and test

In [ ]:
train = pd.merge(train,added_2,on=['card_id'],how='left')
test_final = pd.merge(test_final,added_2,on=['card_id'],how='left')

train = pd.merge(train,added_3,on=['card_id'],how='left')
test_final = pd.merge(test_final,added_3,on=['card_id'],how='left')

train = pd.merge(train,added_4,on=['card_id'],how='left')
test_final = pd.merge(test_final,added_4,on=['card_id'],how='left')


train.fillna(0,inplace=True)
test_final.fillna(0,inplace=True)

## Something intresting about ELO
In Discussions we can found some intrestings thinks about ELO. ([This discussion](https://www.kaggle.com/c/elo-merchant-category-recommendation/discussion/75034))  
[ELO](https://en.wikipedia.org/wiki/Elo_rating_system) - is a method for calculating the relative skill levels of players in zero-sum games such as chess.  
Also we can find formula, to calculate ELO rating:   
$$E_A=\frac{1}{1+10^{(R_B-R_A)/400}}$$  
As we can see from description of this formula, we need 2 veriables, that mean ratings of 2 parametrs.  

We try calculate this between this samples:
- authorized purchase and non authorized purchase
- month_diff and month_lag as most important features  

But I'm not sure that these variables are what we need.

In [ ]:
train['ELO_auth_sum'] = 1/(1+pow(10,(train.authpurchase_amount_sum-train.non_authpurchase_amount_sum)/400))
train['ELO_purchase_sum'] = 1/(1+pow(10,(train.authpurchase_amount_sum+train.non_authpurchase_amount_sum-abs(train.non_authpurchase_amount_sum))/400))

train['ELO_auth_mean'] = 1/(1+pow(10,(train.authpurchase_amount_mean-train.non_authpurchase_amount_mean)/400))
train['ELO_purchase_mean'] = 1/(1+pow(10,(train.authpurchase_amount_mean+train.non_authpurchase_amount_mean-abs(train.non_authpurchase_amount_mean))/400))

In [ ]:
test_final['ELO_auth_sum'] = 1/(1+pow(10,(test_final.authpurchase_amount_sum-test_final.non_authpurchase_amount_sum)/400))
test_final['ELO_purchase_sum'] = 1/(1+pow(10,(test_final.authpurchase_amount_sum+test_final.non_authpurchase_amount_sum-abs(test_final.non_authpurchase_amount_sum))/400))

test_final['ELO_auth_mean'] = 1/(1+pow(10,(test_final.authpurchase_amount_mean-test_final.non_authpurchase_amount_mean)/400))
test_final['ELO_purchase_mean'] = 1/(1+pow(10,(test_final.authpurchase_amount_mean+test_final.non_authpurchase_amount_mean-abs(test_final.non_authpurchase_amount_mean))/400))

In [ ]:
train['ELO_auth_month_diff'] = 1/(1+pow(10,(train.authmonth_diff_mean-train.non_authmonth_diff_mean)/400))
train['ELO_auth_new_month_diff'] = 1/(1+pow(10,(train.authmonth_diff_mean-train.new_month_diff_mean)/400))

train['ELO_auth_month_lag'] = 1/(1+pow(10,(train.authmonth_lag_mean-train.non_authmonth_lag_mean)/400))
train['ELO_auth_new_month_lag'] = 1/(1+pow(10,(train.authmonth_lag_mean-train.new_month_lag_mean)/400))

In [ ]:
test_final['ELO_auth_month_diff'] = 1/(1+pow(10,(test_final.authmonth_diff_mean-test_final.non_authmonth_diff_mean)/400))
test_final['ELO_auth_new_month_diff'] = 1/(1+pow(10,(test_final.authmonth_diff_mean-test_final.new_month_diff_mean)/400))

test_final['ELO_auth_month_lag'] = 1/(1+pow(10,(test_final.authmonth_lag_mean-test_final.non_authmonth_lag_mean)/400))
test_final['ELO_auth_new_month_lag'] = 1/(1+pow(10,(test_final.authmonth_lag_mean-test_final.new_month_lag_mean)/400))

### Calculating rating

(added)  
After few days of thinking and discussing all the things we decided that variables we really need are rating of card and border of loyality (positive or negative).  
Because if we read description of ELO we see, that rating of player (in this competition card) increase after win by fromula:
$$R'_A=R_A+K*(S_A-E_A)$$
where $K$ - koef for top 10 =10, for middle = 20, for first 30 games = 40, $S_A$ - points(1 for win, 0.5 for draw, 0 for loose)  
We have authorized transaction, nonauthorized transaction and positive or negative.  
We split it like this:
- positive authorized transaction like 1
- all nonauthorized transaction like 0   
For $K$ i get only 20 and 40 koef
After that i get feature like rating of card

In [ ]:
rating = pd.concat((hist[['card_id','merchant_id','purchase_amount','purchase_date','month_lag','authorized_flag']],new_t[['card_id','merchant_id','purchase_amount','purchase_date','month_lag','authorized_flag']]))
rating['month_diff'] = ((datetime.datetime.today() - rating['purchase_date']).dt.days)//30
rating['month_diff'] += rating['month_lag']
rating.head()

In [ ]:
rating = rating.sort_values(['card_id','purchase_date'])
rating['num_of_purchase'] = rating.groupby(['card_id']).cumcount()+1

rating['K_koef'] = 0
rating.loc[(rating['authorized_flag']=='Y')&(rating['num_of_purchase']<=30),'K_koef'] = 40
rating.loc[(rating['authorized_flag']=='Y')&(rating['num_of_purchase']>30),'K_koef'] = 20

rating['S_koef'] = 0
rating.loc[(rating['authorized_flag']=='Y'),'S_koef'] = 1

rating['change_rating'] = rating['K_koef'] * (rating['S_koef'] - rating['purchase_amount'])
rating['rating_of_card'] = rating.groupby(['card_id'])['change_rating'].cumsum()

rating['change_rating2'] = rating['K_koef'] * (rating['S_koef'] - rating['month_diff'])
rating['rating_of_card2'] = rating.groupby(['card_id'])['change_rating2'].cumsum()

rating['change_rating3'] = rating['K_koef'] * (rating['S_koef'] - abs(rating['purchase_amount']))
rating['rating_of_card3'] = rating.groupby(['card_id'])['change_rating3'].cumsum()

rating['change_rating4'] = rating['K_koef'] * (rating['S_koef'] - abs(rating['month_diff']))
rating['rating_of_card4'] = rating.groupby(['card_id'])['change_rating4'].cumsum()


It will be rating of card. But we need second rating. We think it would be merchant rating.  
And ELO we can calculate with usage card rating vs merchant rating.   

In [ ]:
rating = rating.sort_values(['merchant_id','purchase_date'])
rating['num_of_purchase'] = rating.groupby(['merchant_id']).cumcount()+1

rating['K_koef'] = 0
rating.loc[(rating['authorized_flag']=='Y')&(rating['num_of_purchase']<=30),'K_koef'] = 40
rating.loc[(rating['authorized_flag']=='Y')&(rating['num_of_purchase']>30),'K_koef'] = 20

rating['S_koef'] = 0
rating.loc[(rating['authorized_flag']=='Y'),'S_koef'] = 1

rating['change_rating_merch'] = rating['K_koef'] * (rating['S_koef'] - (rating['purchase_amount']*(-1)))
rating['rating_of_merch'] = rating.groupby(['merchant_id'])['change_rating_merch'].cumsum()

rating['change_rating_merch2'] = rating['K_koef'] * (rating['S_koef'] - rating['month_diff'])
rating['rating_of_merch2'] = rating.groupby(['merchant_id'])['change_rating_merch2'].cumsum()

rating['change_rating_merch3'] = rating['K_koef'] * (rating['S_koef'] - abs(rating['purchase_amount']))
rating['rating_of_merch3'] = rating.groupby(['merchant_id'])['change_rating_merch3'].cumsum()

rating['change_rating_merch4'] = rating['K_koef'] * (rating['S_koef'] - abs(rating['month_diff']))
rating['rating_of_merch4'] = rating.groupby(['merchant_id'])['change_rating_merch4'].cumsum()

### Calculating ELO

In [ ]:
rating['ELO_1'] = 1/(1+pow(10,(rating.rating_of_card -rating.rating_of_merch )/400))
rating['ELO_2'] = 1/(1+pow(10,(rating.rating_of_card2-rating.rating_of_merch2)/400))
rating['ELO_3'] = 1/(1+pow(10,(rating.rating_of_card3-rating.rating_of_merch3)/400))
rating['ELO_4'] = 1/(1+pow(10,(rating.rating_of_card4-rating.rating_of_merch4)/400))

rating['ELO_11'] = 1/(1+pow(10,(rating.rating_of_card -rating.rating_of_merch )/400))
rating['ELO_21'] = 1/(1+pow(10,(rating.rating_of_card2-rating.rating_of_merch2)/400))
rating['ELO_31'] = 1/(1+pow(10,(rating.rating_of_card3-rating.rating_of_merch3)/400))
rating['ELO_41'] = 1/(1+pow(10,(rating.rating_of_card4-rating.rating_of_merch4)/400))

rating['ELO_12'] = 1/(1+pow(10,(rating.rating_of_card -rating.rating_of_merch )/400))
rating['ELO_22'] = 1/(1+pow(10,(rating.rating_of_card2-rating.rating_of_merch2)/400))
rating['ELO_32'] = 1/(1+pow(10,(rating.rating_of_card3-rating.rating_of_merch3)/400))
rating['ELO_42'] = 1/(1+pow(10,(rating.rating_of_card4-rating.rating_of_merch4)/400))

In [ ]:
#rating = replace_id(rating)
rating_agg = {'S_koef':'mean',
                 'change_rating'   :'mean',
                 'rating_of_card' :'max',
                 'rating_of_card2':'max',
                 'rating_of_card3':'max',
                 'rating_of_card4':'max',
                 'rating_of_merch':'max',
                 'rating_of_merch2':'max',
                 'rating_of_merch3':'max',
                 'rating_of_merch4':'max',
                 'ELO_1':'mean',
                 'ELO_2':'mean',
                 'ELO_3':'mean',
                 'ELO_4':'mean',
                 'ELO_11':'max',
                 'ELO_21':'max',
                 'ELO_31':'max',
                 'ELO_41':'max',
                 'ELO_12':'sum',
                 'ELO_22':'sum',
                 'ELO_32':'sum',
                 'ELO_42':'sum'
             }
rating_card = rating.groupby('card_id').agg(rating_agg).reset_index()
rating_card.head()

In [ ]:
train = pd.merge(train,rating_card,on='card_id',how='left')
test_final = pd.merge(test_final,rating_card,on='card_id',how='left')

train.fillna(0,inplace=True)
test_final.fillna(0,inplace=True)

> And finaly add features, that calculate credit facility of each card

In [ ]:
union = pd.concat((hist,new_t))
union[union.installments==-1]
union = union[union.installments != 0].sort_values(['purchase_date'])
union['installments_lag'] = union.month_lag+union.installments
union['installments_lag2'] = union.month_lag+union.installments
union.loc[union.installments_lag<0,'installments_lag'] = 0
union['credit_now'] = union.installments_lag*union.purchase_amount
union['credit_all'] = union.installments*union.purchase_amount
union2 = union.groupby('card_id').agg({'installments_lag':['min','max','sum','mean','var','median']
                                ,'installments_lag2':['min','max','sum','mean','var','median']
                            ,'credit_now':['min','max','sum','mean','var','median']
                            ,'credit_all':['min','max','sum','mean','var','median']})
union2.columns = ['install_'.join(col).strip() for col in union2.columns.values]
union2.reset_index(inplace=True)
union2.fillna(0,inplace=True)

train = pd.merge(train,union2,on='card_id',how='left')
test_final = pd.merge(test_final,union2,on='card_id',how='left')
train.fillna(0,inplace=True)
test_final.fillna(0,inplace=True)

Saving produced dataset on disk, for clear memory and work only with data

In [ ]:
%%time
train.to_csv(r'F:\Work\RepoSVNwn\Data\train.csv')
test_final.to_csv(r'F:\Work\RepoSVNwn\Data\test_final.csv')

In [ ]:
%%time
train = pd.read_csv(r'F:\Work\RepoSVNwn\Data\train.csv')
test_final = pd.read_csv(r'F:\Work\RepoSVNwn\Data\test_final.csv')

### Drop all high-correlated features

We have really many features (something about 2100)  
Lets cut features, that have hight correlation (not good method, because some features may correlate, but have diffrent information)  
<b>IMPORTANT</b>  
We get TWO train dataframe, one with this block (5.3) and one without this block. After that a just get mean values of two submit

In [ ]:
%%time
print('Search correlations - ',end='')
features = [a for a in train.columns if a not in ['target','first_active_month','card_id']] 
corr_matrix = train[features].corr()
print('ready')
i = 1
ar = []
for a in corr_matrix.index:
    ar.append(i)
    i+=1
corr_matrix['rank'] = ar

In [ ]:
fe_no_corr = []
corr_coef = 0.90
for a in corr_matrix.columns:
    if a == 'rank':
        continue
    r1 = corr_matrix[(corr_matrix.index ==a)]['rank'].values[0]
    try:
        r2 = corr_matrix[(abs(corr_matrix[a]) >= corr_coef)&(corr_matrix.index !=a)]['rank'].values[0]
    except:
        r2 = 100000
    #print(a,'with rank',corr_matrix[(corr_matrix.index ==a)]['rank'].values[0])
    #print('Correlation with',corr_matrix[(corr_matrix[a] >= corr_coef)&(corr_matrix.index !=a)].index.values,
    #     'with ranks:',corr_matrix[(corr_matrix[a] >= corr_coef)&(corr_matrix.index !=a)]['rank'].values)
    if r1<r2:
        fe_no_corr.append(a)
print('='*10)
print('Features is',len(fe_no_corr),':',fe_no_corr)
        

Now we have 469 features. And we will be work with them  
Let's take only 469 features and save our clear dataset

In [ ]:
%%time
f_save = []
for a in fe_no_corr:
    f_save.append(a)
f_save.append('card_id')
f_save.append('target')
f_save.append('first_active_month')
train[f_save].to_csv(r'F:\Work\RepoSVNwn\Data\train_clear.csv')
test_final[[a for a in f_save if a !='target']].to_csv(r'F:\Work\RepoSVNwn\Data\test_final_clear.csv')

In [ ]:
train = train[f_save]
test_final = test_final[[a for a in f_save if a !='target']]

## Training models  

Scheme of training.  

In [ ]:
+-----------------+   +------------------+   +------------------+               +----------------+
|                 |   |                  |   |                  |               |                |
| new_transaction |   | hist_transaction +-->+ auth_transaction |               |   merch_data   |
|                 |   |                  |   |                  |               |                |
+--------+--------+   +--------+---------+   +----+-------------+               +--------+-------+
         |                     |                  |                                      |
         |                     v                  |                                      |
         |            +--------+---------+        |                                      |
         |            |                  |        |                                      |
         |            |noauth_transaction|        |                                      |
         |            |                  |        |                                      |
         |            +--------+---------+        v                                      |
         |                     |       +----------+------+                               |
         |                     +------>+                 |                               |
         |                             |   train_data    +<------------------------------+
         +---------------------------->+ ~ 2500 features |
                                       +---------+-------+
                                                 |
                                                 |
                                                 v
                                          +------+------+
                                          |             |
                                          |    lgbm     |
                                          |             |
                                          +------+------+
                                                 |
                                                 v
                                +----------------+-----------------+
                                |                                  |
                                | FI best [50,100,150,200,300,400] |
                                |                                  |
                                +----------------+-----------------+
                                                 |
                                                 |
                                                 |
                                                 |
        +---------------+---------------+--------+--------+---------------+---------------+
        |               |               |                 |               |               |
        v               v               v                 v               v               v
 +------+------+ +------+------+ +------+------+   +------+------+ +------+------+ +------+------+
 |  K-fold 10  | |  K-fold 10  | |  K-fold 10  |   |  K-fold 10  | |  K-fold 10  | |  K-fold 10  |
 |  goss lgbm  | |  gbdt lgbm  | |  dart lgbm  |   |  lgbm tuned | |  lgbm tuned | |  lgbm tuned |
 |             | |             | |             |   |     goss    | |     dart    | |     gbrt    |
 +------+------+ +------+------+ +----------+--+   +--+----------+ +------+------+ +------+------+
        |               |                   |         |                   |               |
        |               |                   |         |                   |               |
        |               |                   v         v                   |               |
        |               |               +---+---------+----+              |               |
        |               +-------------->+                  +<-------------+               |
        |                               |   ByesianRidge   |                              |
        +------------------------------>+                  +<-----------------------------+
                                        +--------+---------+
                                                 |
                                                 v
                                        +--------+---------+
                                        |     Submit       |
                                        | param:outlier if |
                                        |   target <-15    |
                                        +------------------+


*block for load data if notebook was shutdown or reload*

In [ ]:
%%time
train = pd.read_csv('../input/train.csv')
train_cards = train['card_id']
train = pd.read_csv('../input/train_clear.csv')
train['card_id'] = train_cards
test_final = pd.read_csv('../input/test_final_clear.csv')

*end load block*

In [ ]:
cards = test_final[['card_id']]
train['outlier'] = 0
train.loc[train.target<-22.,'outlier'] = 1
outlier = train['outlier']

In [ ]:
#train.target = deanon_target(train.target)
target = train.target

![](http://)![](http://)Little restruct features.

In [ ]:
features = [a for a in train.columns if a not in ['target','first_active_month','card_id']] 
purchase_column = [a for a in features if 'purchase_amount' in a]
for a in [a for a in features if 'rating' in a]:
    purchase_column.append(a)

In [ ]:
for a in purchase_column:
    train['anon_'+a] = anon_target(train[a])
    test_final['anon_'+a] = anon_target(test_final[a])
print('Done')

### Zero model: Predict on all/no correlate features
First of all I train model to take good features, that can be usefull. I take 400 best features.  
Also we save prediction, to use it in future mix.

In [ ]:
features = [a for a in train.columns if a not in ['card_id','target','first_active_month','outlier','p1','p1_5','p2_5','pred_valid','pred_valid2','pred_valid3']]

In [ ]:
%%time 
cat_features = ['feature_1', 'feature_2', 'feature_3']

folds = KFold(n_splits=5, shuffle=True, random_state=15)
oof = np.zeros(len(train))
predictions = np.zeros(len(test_final))
start = time.time()

print('ready')

for fold_, (trn_idx, val_idx) in enumerate(folds.split(train[features].values, target.values)):
    param = {'task': 'train',
        'boosting': 'gbdt',
        'objective': 'regression',
        'metric': 'rmse',
        'learning_rate': 0.01,
        'subsample': 0.9855232997390695,
        'max_depth': 7,
        'top_rate': 0.9064148448434349,
        'num_leaves': 20,
        'min_child_weight': 41.9612869171337,
        'other_rate': 0.0721768246018207,
        'reg_alpha': 9.677537745007898,
        'colsample_bytree': 0.5665320670155495,
        'min_split_gain': 9.820197773625843,
        'reg_lambda': 8.2532317400459,
        'min_data_in_leaf': 21,
        'verbose': -1,
        'seed':int(2**fold_*2),
        'bagging_seed':int(2**fold_*2),
        'drop_seed':int(2**fold_*2),
        'lambda_l2':5,
        'lambda_l1':5,
        "feature_fraction": 0.85,
        "bagging_freq": 1,
        "bagging_fraction": 0.9 ,
        "bagging_seed": int(2**fold_*2),
        "max_bin":4,
        "n_jobs":6
        }
    print("fold n°{}".format(fold_))
    trn_data = lgb.Dataset(train.iloc[trn_idx][features],
                           label=target.iloc[trn_idx],
                           categorical_feature=cat_features
                          )
    val_data = lgb.Dataset(train.iloc[val_idx][features],
                           label=target.iloc[val_idx],
                           categorical_feature=cat_features
                          )
    print('Data ready')
    num_round = 10000
    clf = lgb.train(param,
                    trn_data,
                    num_round,
                    valid_sets = [trn_data, val_data],
                    verbose_eval=100,
                    early_stopping_rounds = 200)
    
    oof[val_idx] = clf.predict(train.iloc[val_idx][features], num_iteration=clf.best_iteration)
    
    predictions += clf.predict(test_final[features], num_iteration=clf.best_iteration) / folds.n_splits

print("CV score: {:<8.5f}".format(mean_squared_error(oof, target)**0.5))
#print("CV score: {:<8.5f}".format(mean_squared_error(anon_target(np.where(oof<0,0.0000001,oof)), anon_target(np.where(target<0,0.0000001,target)))**0.5))

[](http://)Saving this predict

In [ ]:
dictionary = cards
dictionary['target'] = predictions
#dictionary.loc[dictionary['target']<-12,'target'] = -33.21928095
dictionary[['card_id','target']].to_csv('/all_feature_210219.csv',index=False)
dictionary[['card_id','target']].head()

In [ ]:
fi2 = pd.DataFrame(clf.feature_importance())
fi2.columns = ['importance']
fi2['feature'] = features

plt.figure(figsize=(14,25))
sns.barplot(x="importance",
            y="feature",
            data=fi2.sort_values(by="importance",
                                           ascending=False)[:100])

#### Analysis predict

Lets see, what of features can be category features

In [ ]:
%%time
cat_col = []
for a in train.columns:
    if train[a].nunique() < 5:
        if a != 'outlier':
            cat_col.append(a)
cat_col

### First model: LBGM on goss

Fit main model on all data  
Using K-fold with 8 splits and enabled shuffle + using diffrent count of features, selected by feature importance

In [ ]:
%%time
oof = []
predictions = []


for a in [50,100,150,200,300,400]:
    oof.append(np.zeros(len(train)))
    predictions.append(np.zeros(len(test_final)))
    
i=0
for a in [50,100,150,200,300,400]:                       
    features = fi2.sort_values(by="importance",ascending=False)['feature'][:a]
    cat_features = []

    folds = KFold(n_splits=8, shuffle=True, random_state=a*2)
    start = time.time()
    print('ready')

    for fold_, (trn_idx, val_idx) in enumerate(folds.split(train[features].values, target.values)):
        print("fold n°{}".format(fold_))
        param = {'task': 'train',
            'boosting': 'goss',
            'objective': 'regression',
            'metric': 'rmse',
            'learning_rate': 0.005,
            'subsample': 0.9855232997390695,
            'max_depth': 7,
            'top_rate': 0.9064148448434349,
            'num_leaves': 63,
            'min_child_weight': 41.9612869171337,
            'other_rate': 0.0721768246018207,
            'reg_alpha': 9.677537745007898,
            'colsample_bytree': 0.5665320670155495,
            'min_split_gain': 9.820197773625843,
            'reg_lambda': 8.2532317400459,
            'min_data_in_leaf': 21,
            'verbose': -1,
            'seed':int(2**fold_*a),
            'bagging_seed':int(2**fold_/a),
            'drop_seed':int(2**fold_+a),
            "n_jobs":20,
            "lambda_l1": 0.05,
            }
        trn_data = lgb.Dataset(train.iloc[trn_idx][features],
                               label=target.iloc[trn_idx],
                               categorical_feature=cat_features
                              )
        val_data = lgb.Dataset(train.iloc[val_idx][features],
                               label=target.iloc[val_idx],
                               categorical_feature=cat_features
                              )
        print('Data ready')
        num_round = 10000
        clf = lgb.train(param,
                        trn_data,
                        num_round,
                        valid_sets = [trn_data, val_data],
                        verbose_eval=100,
                        #feval = minowski_dist,
                        early_stopping_rounds = 200)

        oof[i][val_idx] = clf.predict(train.iloc[val_idx][features], num_iteration=clf.best_iteration)

        predictions[i] += clf.predict(test_final[features], num_iteration=clf.best_iteration) / folds.n_splits

    print("CV score: {:<8.5f}".format(mean_squared_error(oof[i], target)**0.5))
    train['p1_'+str(a)] = oof[i]
    i+=1
result = np.zeros(len(train))
for a in range(0,len(oof)):
    result += oof[a]/len(oof)
print("CV score: {:<8.5f}".format(mean_squared_error(result, target)**0.5))

### Second model: Dart
Fit main model on all data  
Using K-fold with 3 splits and enabled shuffle + using diffrent count of features, selected by feature importance  
Get only 3 k-fold just because dart calculate can spend many time

In [ ]:
%%time
oof2 = []
predictions2 = []

for a in [50,100,150,200,300,400]:
    oof2.append(np.zeros(len(train)))
    predictions2.append(np.zeros(len(test_final)))
    
i=0
for a in [50,100,150,200,300,400]:                       
    features = fi2.sort_values(by="importance",ascending=False)['feature'][:a]
    cat_features = []

    folds = KFold(n_splits = 3,shuffle = True, random_state = 12*a)
    start = time.time()

    param = {'num_leaves': 31,
              'min_data_in_leaf': 32, 
              'objective':'regression',
              'max_depth': -1,
              'learning_rate': 0.004,
              "min_child_samples": 20,
              "boosting": "dart",
              "feature_fraction": 0.9,
              "bagging_freq": 1,
              "bagging_fraction": 0.9 ,
              "bagging_seed": 11,
              "metric": 'rmse',
              "lambda_l1": 0.18,
              "nthread": 24,
              "verbosity": -1}
    print('ready')
    for fold_, (trn_idx, val_idx) in enumerate(folds.split(train[features].values, target.values)):
        print("fold n°{}".format(fold_))
        trn_data = lgb.Dataset(train.iloc[trn_idx][features],
                               label=target.iloc[trn_idx],
                               categorical_feature=cat_features
                              )
        val_data = lgb.Dataset(train.iloc[val_idx][features],
                               label=target.iloc[val_idx],
                               categorical_feature=cat_features
                              )
        print('Data ready')
        num_round = 10000
        clf = lgb.train(param,
                        trn_data,
                        num_round,
                        valid_sets = [trn_data, val_data],
                        verbose_eval=100,
                        #feval = minowski_dist,
                        early_stopping_rounds = 200)

        oof2[i][val_idx] = clf.predict(train.iloc[val_idx][features], num_iteration=clf.best_iteration)

        predictions2[i] += clf.predict(test_final[features], num_iteration=clf.best_iteration) / folds.n_splits

    print("CV score: {:<8.5f}".format(mean_squared_error(oof2[i], target)**0.5))
    train['p2_'+str(a)] = oof2[i]
    i+=1
    
result2 = np.zeros(len(train))
for a in range(0,len(oof2)):
    result2 += oof2[a]/len(oof2)
print("CV score: {:<8.5f}".format(mean_squared_error(result2, target)**0.5))

In [ ]:
dictionary = cards
dictionary['target'] = predictions
dictionary[['card_id','target']].to_csv('/dart_150219_cv_3.65983.csv',index=False)
dictionary[['card_id','target']].head()

### Third model: gbdt
Fit main model on all data  
Using K-fold with 3 splits and enabled shuffle + using diffrent count of features, selected by feature importance  

In [ ]:
%%time
oof3 = []
predictions3 = []

for a in [50,100,150,200,300,400]:
    oof3.append(np.zeros(len(train)))
    predictions3.append(np.zeros(len(test_final)))
    
i=0
for a in [50,100,150,200,300,400]:                       
    features = fi2.sort_values(by="importance",ascending=False)['feature'][:a]
    cat_features = []

    folds = KFold(n_splits = 7,shuffle = True, random_state = 12*a)
    start = time.time()

    print('ready')

    for fold_, (trn_idx, val_idx) in enumerate(folds.split(train[features].values, target.values)):
        print("fold n°{}".format(fold_))
        param = {'num_leaves': 10,
             'min_data_in_leaf': 32, 
             'objective':'regression',
             'max_depth': -1,
             'learning_rate': 0.005,
             "min_child_samples": 20,
             "boosting": "gbdt",
             "feature_fraction": 0.9,
             "bagging_freq": 1,
             "bagging_fraction": 0.9 ,
             "bagging_seed": int(2**fold_*a),
             "metric": 'rmse',
             "lambda_l1": 0.11,
             "nthread": 10,
             "verbosity": -1}
        trn_data = lgb.Dataset(train.iloc[trn_idx][features],
                               label=target.iloc[trn_idx],
                               categorical_feature=cat_features
                              )
        val_data = lgb.Dataset(train.iloc[val_idx][features],
                               label=target.iloc[val_idx],
                               categorical_feature=cat_features
                              )
        print('Data ready')
        num_round = 10000
        clf = lgb.train(param,
                        trn_data,
                        num_round,
                        valid_sets = [trn_data, val_data],
                        verbose_eval=100,
                        #feval = minowski_dist,
                        early_stopping_rounds = 200)

        oof3[i][val_idx] = clf.predict(train.iloc[val_idx][features], num_iteration=clf.best_iteration)

        predictions3[i] += clf.predict(test_final[features], num_iteration=clf.best_iteration) / folds.n_splits

    print("CV score: {:<8.5f}".format(mean_squared_error(oof3[i], target)**0.5))
    train['p3_'+str(a)] = oof3[i]
    i+=1
result3 = np.zeros(len(train))
for a in range(0,len(oof3)):
    result3 += oof3[a]/len(oof3)
print("CV score: {:<8.5f}".format(mean_squared_error(result3, target)**0.5))

In [ ]:
dictionary = cards
dictionary['target'] = predictions
dictionary[['card_id','target']].to_csv('/gbdt_150219_cv_3.65867.csv',index=False)
dictionary[['card_id','target']].head()

### Fourth model: Tuned LGBM gbrt ([Dmitry Kravchuk](https://www.kaggle.com/dishkakrauch))
Using K-fold with 5 splits and enabled shuffle + using diffrent count of features, selected by feature importance  

In [ ]:
%%time
oof5 = []
predictions5 = []

for a in [50,100,150,200,300,400]:
    oof5.append(np.zeros(len(train)))
    predictions5.append(np.zeros(len(test_final)))
    
i=0
for a in [50,100,150,200,300,400]:                       
    features = fi2.sort_values(by="importance",ascending=False)['feature'][:a]
    cat_features = []

    folds = KFold(n_splits=5, shuffle=True, random_state=a*2)
    start = time.time()
    print('ready')

    for fold_, (trn_idx, val_idx) in enumerate(folds.split(train[features].values, target.values)):
        print("fold n°{}".format(fold_))
        param = {
        #Core Parameters
            'task': 'train',
            'objective': 'regression',
            'boosting': 'gbrt',
            'learning_rate': .01,
            'num_leaves': 114, #2^max_depth*70%
            'num_threads': 24,
            'device_type': 'cpu',
            'seed': int(2**a*fold_),
        #Learning Control Parameters
            'max_depth': 3,
            'min_data_in_leaf': 20,
            'min_sum_hessian_in_leaf': 1e-3,
            'bagging_fraction': .75,
            'bagging_freq': 2,
            'bagging_seed': int(2**a*fold_),
            'feature_fraction': .75,
            'feature_fraction_seed': int(2**a*fold_),
            'max_delta_step': 0.0,
            'lambda_l1': 170.,
            'lambda_l2': 227.,
            'min_gain_to_split': 0.0,
            'min_data_per_group': 100,
            'max_cat_threshold': 32,
            'cat_l2': 10.0,
            'cat_smooth': 10.0,
            'max_cat_to_onehot': 4,
            'monotone_constraints': None,
            'feature_contri': None,
            'forcedsplits_filename': '',
            'refit_decay_rate': .9,
            'verbosity': 1,
            'max_bin': 171,
            'min_data_in_bin': 3,
            'bin_construct_sample_cnt': 200000,
            'histogram_pool_size': -1.0,
            'data_random_seed': int(2**a*fold_),
            'snapshot_freq': -1,
            'iniscore_filename': '',
            'valid_data_iniscore': '',
            'pre_partition': False,
            'enable_bundle': True,
            'max_conflict': 0.0,
            'is_enable_sparse': True,
            'sparse_threshold': .8,
            'use_missing': True,
            'zero_as_missing': True,
            'two_round': False,
            'save_binary': False,
            'enable_load_from_binary_file': True,
            'header': False,
            'label_column': '',
            'weight_columns': '',
            'group_column': '',
            'ignore_columns': '',
            'boost_from_average': True,
            'reg_sqrt': False,
            'alpha': .9,
            'fair_c': 1.0,
            'poisson_max_delta_step': .7,
            'tweedie_variance_power': 1.5,
        #Metric Parameters
            'metric': 'l2_root',
            'metric_freq': 1,
            'is_provide_training_metric': False
        }
        trn_data = lgb.Dataset(train.iloc[trn_idx][features],
                               label=target.iloc[trn_idx],
                               categorical_feature=cat_features
                              )
        val_data = lgb.Dataset(train.iloc[val_idx][features],
                               label=target.iloc[val_idx],
                               categorical_feature=cat_features
                              )
        print('Data ready')
        num_round = 10000
        clf = lgb.train(param,
                        trn_data,
                        num_round,
                        valid_sets = [trn_data, val_data],
                        verbose_eval=100,
                        #feval = minowski_dist,
                        early_stopping_rounds = 200)

        oof5[i][val_idx] = clf.predict(train.iloc[val_idx][features], num_iteration=clf.best_iteration)

        predictions5[i] += clf.predict(test_final[features], num_iteration=clf.best_iteration) / folds.n_splits

    print("CV score: {:<8.5f}".format(mean_squared_error(oof5[i], target)**0.5))
    train['p5_'+str(a)] = oof5[i]
    i+=1
result = np.zeros(len(train))
for a in range(0,len(oof5)):
    result += oof5[a]/len(oof5)
print("CV score: {:<8.5f}".format(mean_squared_error(result, target)**0.5))

### Fifth model: Tuned LGBM goss (Krauchuk_D)
Using K-fold with 5 splits and enabled shuffle + using diffrent count of features, selected by feature importance  

In [ ]:
%%time
oof6 = []
predicions6 = []

for a in [50,100,150,200,300,400]:
    oof6.append(np.zeros(len(train)))
    predicions6.append(np.zeros(len(test_final)))
    
i=0
for a in [50,100,150,200,300,400]:                       
    features = fi2.sort_values(by="importance",ascending=False)['feature'][:a]
    cat_features = []

    folds = KFold(n_splits=5, shuffle=True, random_state=a*2)
    start = time.time()
    print('ready')

    for fold_, (trn_idx, val_idx) in enumerate(folds.split(train[features].values, target.values)):
        print("fold n°{}".format(fold_))
        param = {
        #Core Parameters
            'task': 'train',
            'objective': 'regression',
            'boosting': 'goss',
            'learning_rate': .01,
            'num_leaves': 171, #2^max_depth*70%
            'num_threads': 24,
            'device_type': 'cpu',
            'seed': int(2**a*fold_),
        #Learning Control Parameters
            'max_depth': 3,
            'min_data_in_leaf': 20,
            'min_sum_hessian_in_leaf': 1e-3,
            'feature_fraction': .75,
            'feature_fraction_seed': int(2**a*fold_),
            'max_delta_step': 0.0,
            'lambda_l1': 114.,
            'lambda_l2': 199.,
            'min_gain_to_split': 0.0,
            'top_rate': .4, #goss
            'oter_rate': .4, #goss
            'min_data_per_group': 100,
            'max_cat_threshold': 32,
            'cat_l2': 10.0,
            'cat_smooth': 10.0,
            'max_cat_to_onehot': 4,
            'monotone_constraints': None,
            'feature_contri': None,
            'forcedsplits_filename': '',
            'refit_decay_rate': .9,
        #IO Parameters
            'verbosity': 1,
            'max_bin': 171,
            'min_data_in_bin': 3,
            'bin_construct_sample_cnt': 200000,
            'histogram_pool_size': -1.0,
            'data_random_seed': int(2**a*fold_),
            'snapshot_freq': -1,
            'iniscore_filename': '',
            'valid_data_iniscore': '',
            'pre_partition': False,
            'enable_bundle': True,
            'max_conflict': 0.0,
            'is_enable_sparse': True,
            'sparse_threshold': .8,
            'use_missing': True,
            'zero_as_missing': True,
            'two_round': False,
            'save_binary': False,
            'enable_load_from_binary_file': True,
            'header': False,
            'label_column': '',
            'weight_columns': '',
            'group_column': '',
            'ignore_columns': '',
            'boost_from_average': True,
            'reg_sqrt': False,
            'alpha': .9,
            'fair_c': 1.0,
            'poisson_max_delta_step': .7,
            'tweedie_variance_power': 1.5,
        #Metric Parameters
            'metric': 'l2_root',
            'metric_freq': 1,
            'is_provide_training_metric': False,
        #Network Parameters
            'num_machines': 1,
            'local_listen_port': 12400,
            'time-out': 120,
            'machine_list_filename': '',
            'machines': '',
        #GPU Parameters
            'gpu_platform_id': 0,
            'gpu_device_id': 0,
            'gpu_use_dp': True
        }
        
        trn_data = lgb.Dataset(train.iloc[trn_idx][features],
                               label=target.iloc[trn_idx],
                               categorical_feature=cat_features
                              )
        val_data = lgb.Dataset(train.iloc[val_idx][features],
                               label=target.iloc[val_idx],
                               categorical_feature=cat_features
                              )
        print('Data ready')
        num_round = 10000
        clf = lgb.train(param,
                        trn_data,
                        num_round,
                        valid_sets = [trn_data, val_data],
                        verbose_eval=100,
                        #feval = minowski_dist,
                        early_stopping_rounds = 200)

        oof6[i][val_idx] = clf.predict(train.iloc[val_idx][features], num_iteration=clf.best_iteration)

        predicions6[i] += clf.predict(test_final[features], num_iteration=clf.best_iteration) / folds.n_splits

    print("CV score: {:<8.5f}".format(mean_squared_error(oof6[i], target)**0.5))
    train['p6_'+str(a)] = oof6[i]
    i+=1
result = np.zeros(len(train))
for a in range(0,len(oof6)):
    result += oof6[a]/len(oof6)
print("CV score: {:<8.5f}".format(mean_squared_error(result, target)**0.5))

### Sixth model: Tuned LGBM dart ([Dmitry Kravchuk](https://www.kaggle.com/dishkakrauch))
Using K-fold with 5 splits and enabled shuffle + using diffrent count of features, selected by feature importance  

In [ ]:
%%time
oof7 = []
predicions7 = []

for a in [50,100,150,200,300,400]:
    oof7.append(np.zeros(len(train)))
    predicions7.append(np.zeros(len(test_final)))
    
i=0
for a in [50,100,150,200,300,400]:                       
    features = fi2.sort_values(by="importance",ascending=False)['feature'][:a]
    cat_features = []

    folds = KFold(n_splits=5, shuffle=True, random_state=a*2)
    start = time.time()
    print('ready')

    for fold_, (trn_idx, val_idx) in enumerate(folds.split(train[features].values, target.values)):
        print("fold n°{}".format(fold_))
        param = {
        #Core Parameters
            'task': 'train',
            'objective': 'regression',
            'boosting': 'dart',
            'learning_rate': .1,
            'num_leaves': 114, #2^max_depth*70%
            'num_threads': 24,
            'device_type': 'cpu',
            'seed': int(2**a*fold_),
        #Learning Control Parameters
            'max_depth': 3,
            'min_data_in_leaf': 20,
            'min_sum_hessian_in_leaf': 1e-3,
            'bagging_fraction': .75,
            'bagging_freq': 2,
            'bagging_seed': int(2**a*fold_),
            'feature_fraction': .75,
            'feature_fraction_seed': int(2**a*fold_),
            'max_delta_step': 0.0,
            'lambda_l1': 0.,
            'lambda_l2': 114.,
            'min_gain_to_split': 0.0,
            'drop_rate': .55, #dart
            'max_dop': 64, #dart
            'skip_drop': .1, #dart
            'xgboost_dart_mode': False, #dart
            'uniform_drop': True, #dart
            'drop_seed': int(2**a*fold_), #dart
            'min_data_per_group': 100,
            'max_cat_threshold': 32,
            'cat_l2': 10.0,
            'cat_smooth': 10.0,
            'max_cat_to_onehot': 4,
            'monotone_constraints': None,
            'feature_contri': None,
            'forcedsplits_filename': '',
            'refit_decay_rate': .9,
        #IO Parameters
            'verbosity': 1,
            'max_bin': 255,
            'min_data_in_bin': 3,
            'bin_construct_sample_cnt': 200000,
            'histogram_pool_size': -1.0,
            'data_random_seed': int(2**a*fold_),
            'snapshot_freq': -1,
            'iniscore_filename': '',
            'valid_data_iniscore': '',
            'pre_partition': False,
            'enable_bundle': True,
            'max_conflict': 0.0,
            'is_enable_sparse': True,
            'sparse_threshold': .8,
            'use_missing': True,
            'zero_as_missing': True,
            'two_round': False,
            'save_binary': False,
            'enable_load_from_binary_file': True,
            'header': False,
            'label_column': '',
            'weight_columns': '',
            'group_column': '',
            'ignore_columns': '',
            'boost_from_average': True,
            'reg_sqrt': False,
            'alpha': .9,
            'fair_c': 1.0,
            'poisson_max_delta_step': .7,
            'tweedie_variance_power': 1.5,
        #Metric Parameters
            'metric': 'l2_root',
            'metric_freq': 1,
            'is_provide_training_metric': False,
        #Network Parameters
            'num_machines': 1,
            'local_listen_port': 12400,
            'time-out': 120,
            'machine_list_filename': '',
            'machines': '',
        #GPU Parameters
            'gpu_platform_id': 0,
            'gpu_device_id': 0,
            'gpu_use_dp': True
        }
        trn_data = lgb.Dataset(train.iloc[trn_idx][features],
                               label=target.iloc[trn_idx],
                               categorical_feature=cat_features
                              )
        val_data = lgb.Dataset(train.iloc[val_idx][features],
                               label=target.iloc[val_idx],
                               categorical_feature=cat_features
                              )
        print('Data ready')
        num_round = 10000
        clf = lgb.train(param,
                        trn_data,
                        num_round,
                        valid_sets = [trn_data, val_data],
                        verbose_eval=100,
                        #feval = minowski_dist,
                        early_stopping_rounds = 200)

        oof7[i][val_idx] = clf.predict(train.iloc[val_idx][features], num_iteration=clf.best_iteration)

        predicions7[i] += clf.predict(test_final[features], num_iteration=clf.best_iteration) / folds.n_splits

    print("CV score: {:<8.5f}".format(mean_squared_error(oof7[i], target)**0.5))
    train['p1_'+str(a)] = oof7[i]
    i+=1
result = np.zeros(len(train))
for a in range(0,len(oof7)):
    result += oof7[a]/len(oof7)
print("CV score: {:<8.5f}".format(mean_squared_error(result, target)**0.5))

### Mix with BayesianRidge
For the best mix we use BayesianRidge and ensemble out models

In [ ]:
train_stack = []
test_stack = []
for a in range(0,len(oof)):
    train_stack.append(oof[a])
    train_stack.append(oof2[a])
    train_stack.append(oof3[a])
    
    train_stack.append(oof5[a])
    train_stack.append(oof6[a])
    train_stack.append(oof7[a])
    
    test_stack.append(predictions[a])
    test_stack.append(predictions2[a])
    test_stack.append(predictions3[a])
    
    test_stack.append(predictions5[a])
    test_stack.append(predicions6[a])
    test_stack.append(predicions7[a])
#train_stack.append(oof4)
#test_stack.append(predictions4)

train_stack = np.vstack(train_stack).transpose()
test_stack = np.vstack(test_stack).transpose()

folds = KFold(n_splits=20,shuffle=True,random_state=4520)
oof_stack = np.zeros(train_stack.shape[0])
predictions_stack = np.zeros(test_stack.shape[0])

for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_stack, target)):
    print("fold n°{}".format(fold_))
    trn_data, trn_y = train_stack[trn_idx], target.iloc[trn_idx].values
    val_data, val_y = train_stack[val_idx], target.iloc[val_idx].values

    print("-" * 10 + "Stacking " + str(fold_) + "-" * 10)
    clf = BayesianRidge()
    clf.fit(trn_data, trn_y)
    
    oof_stack[val_idx] = clf.predict(val_data)
    print("CV score fold: {:<8.5f}".format(mean_squared_error(oof_stack[val_idx], target[val_idx])**0.5))
    predictions_stack += clf.predict(test_stack) / folds.n_splits

print("CV score summary:",np.sqrt(mean_squared_error(target.values, oof_stack)))

In [ ]:
dictionary = cards
dictionary['target'] = predictions_stack
border = -14.5
print('Count outliers:',len(dictionary[dictionary.target<border]))
dictionary[['card_id','target']].to_csv('../stuck_models_no_cat_220219_cv_'+str(round(np.sqrt(mean_squared_error(target.values, oof_stack)),5))+'.csv',index=False)
dictionary[['card_id','target']].head()